# Peru SISepuede Article 6 – Full Tutorial Notebook

This notebook is designed to help you and other users generate simulations to support Article 6 analysis:

- Understand the overall workflow used to construct, transform, and run SISepuede inputs for Peru
- See how directory paths, data files, and model attributes are configured
- Follow how the Peru input dataframe is built from pipeline output and example data
- Learn how the Transformers / Transformations / Strategies machinery is initialized
- See where helper CSVs (inputs and attributes) are written for cloud/batch use
- Understand how the high-level `SISEPUEDEModels` runner is constructed and used
- Inspect additional exploratory, validation, and debugging cells in context

You can safely run the notebook from top to bottom.  
When turning this into a streamlined workflow later, you can focus on the cells marked as
*core workflow* and consider removing or skipping those marked as primarily *exploratory*.

## How to Adapt This Notebook for New Runs

Because the underlying SISepuede logic does **not** change, adapting this notebook for
new countries, datasets, or scenarios mainly involves adjusting:

---

### A. File Paths and Environment Setup

Update paths in the early setup cells:

- **`path_git`**  
  (location of your SISepuede repositories)

- **Data file paths:**  
  - `path_data_calib`  
  - `path_data_base`  
  - `path_data_built`  
  - Repository paths (if used)

These must point to the correct locations for your machine and your region.

---

### B. Country / Region Configuration

To adapt the workflow for a different country:

- Change ISO codes from `"PER"` to the appropriate ISO-3 (e.g., `"TZA"`).
- Update any hard-coded region identifiers or filters.
- Confirm that the input file contains only the regions you expect.

The rest of the logic stays the same.

---

### C. Reusing the Transformation Pattern

Use the example in **cell 24** (the `transformation_general` example) as a template.

To define new levers, modify:

- **Model variable**  
- **Categories**  
- **Magnitude** and **magnitude_type**  
- **Ramp / implementation vector**  
- **Strategy ID**

The structure does not change — only the contents.

---

### D. Export New Helper Files

To prepare your environment for external runs:

- Reuse the export logic that writes  
  - `attribute_dim_time_period.csv`  
  - `input_data_peru_base.csv`

When adapting to a new country, these files will contain **your region’s attributes and input data**.

---

### E. Run Models and Compare Scenarios

Use the existing examples as patterns:

- **AFOLU runs**  
  to sanity-check sector-specific behavior

- **Full SISepuede runs**  
  to evaluate complete scenarios

For strategy runs:

- Tie together your **strategies** (from the transformations directory)
- Plug in transformed inputs or use a **scenario manager**
- Compare outputs across scenarios to understand impacts

---

By following these steps — adjusting paths, region codes, transformations, exports, and run calls — you can reuse this entire template for **any SISepuede region** or **new Article 6 / policy scenarios**.


## 1. Environment and imports

The next cell sets up the Python environment so that SISepuede, the data pipeline,
and local helper modules can be imported. It also adds the relevant git repository
paths to `sys.path` and suppresses non-critical warnings.


In [1]:
# Tutorial note: Environment and imports for SISepuede and helpers. 
# add path to sispeuede to sys.path in python
import sys
import pathlib
import warnings
warnings.filterwarnings("ignore")

import importlib
import matplotlib.pyplot as plt
import numpy as np
import os, os.path
import pandas as pd
import sisepuede.core.support_classes as sc
import sisepuede.manager.sisepuede_examples as sxl
import sisepuede.manager.sisepuede_file_structure as sfs
import sisepuede.manager.sisepuede_models as sm
import sisepuede.transformers as trf
import sisepuede.transformers.transformations as tmts
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf
import sisepuede.visualization.plots as spp
import time
from typing import *

importlib.reload(tmts.trs)




ModuleNotFoundError: No module named 'sisepuede'

In [3]:

path_data_built = pathlib.Path(
    os.path.abspath("../generic_regions/data/output/sisepuede_raw_inputs_latest_PER.csv")
)

## 2. Project directories, file paths, and SISepuede metadata

The upcoming cells define:

- Working directories (`data/`, `transformations_peru/`)
- Paths to Peru-specific calibration and pipeline-built input files
- The SISepuede file structure and model attribute objects
- Region and time-period helpers used throughout the workflow


In [4]:
### SETUP DIRECTORIES

path_cur = pathlib.Path(os.getcwd())
path_data = path_cur.joinpath("data")
path_transformations = path_cur.joinpath("transformations_peru")


##  SETUP FILES

path_data_calib = path_data.joinpath("input_all_sectors_peru.csv")
path_data_base = path_data.joinpath("real_data_TZA_PER_LKA_20250210.csv")
#path_data_built = path_data.joinpath("sisepuede_raw_global_inputs_peru.csv")
path_data_built = pathlib.Path(
    os.path.abspath("../generic_regions/data/output/sisepuede_raw_inputs_latest_PER.csv")
)

##  SETUP SOME PIPELINE STUFF


file_struct = sfs.SISEPUEDEFileStructure()

# assign some shortcuts
matt = file_struct.model_attributes
regions = sc.Regions(matt)
time_periods = sc.TimePeriods(matt)


##  SOME GLOBALS

_REGION_NAME = "peru"
_REGION_ISO = regions.return_region_or_iso(_REGION_NAME, return_type = "iso", )


## 3. Building the Peru input dataframe from pipeline output

The following cells read pipeline-built SISepuede inputs for Peru, attach the
region name, filter to valid time periods, and merge in any fields that appear
in the SISepuede example inputs but are missing from the pipeline output.
The result is a `df_peru` dataframe suitable for running SISepuede models.

## What a new user needs to know (about building `df_peru`)

The end product of this section is a Peru input dataframe (`df_peru` or `df_peru_new`) that is:

- **Filtered to Peru**
- **Harmonized** across older SISepuede inputs and pipeline-built inputs
- **Ready for use** with SISepuede models

If another user wants to repeat this process for a different country:

- They will mainly change:
  - The ISO-3 filter (e.g., `"PER"` → `"TZA"`)
  - The input file paths
- The **pattern of merging, harmonizing, and completing missing fields** is the same regardless of country.


In [26]:

df_peru = pd.read_csv(path_data_built)
df_peru[regions.key] = _REGION_NAME
df_peru = (
    df_peru[
        df_peru[time_periods.field_time_period].isin(
            time_periods.all_time_periods
        )
    ]
    .drop(
        columns = [time_periods.field_year, regions.field_iso], 
    )
    .reset_index(drop = True, )
)





True

## 4. AFOLU sector validation (optional but informative)

The next few cells demonstrate how to run the AFOLU model on the Peru inputs
and inspect livestock-related outputs. These cells are primarily for validation
and understanding sector behavior, and are not strictly required for the core
Article 6 workflow.

### What a New User Needs to Know

- These cells are **validation**, not scenario-building.  
- They show how to run and test a **single SISepuede sector model** (e.g., AFOLU) against newly built inputs.
- Only modify these cells if:
  - You want to validate a **different region’s** input dataframe, or  
  - You need to inspect **different variables or sectors**.


In [ ]:
# Tutorial note: Run AFOLU sector model for validation and exploration. 
import sisepuede.models.afolu as mafl
model_afolu = mafl.AFOLU(matt, )
df_outg = model_afolu(df_peru, )


# plot 
fig, ax = plt.subplots(figsize = (13, 7))
svp.plot_emissions_stack(
    df_out,
    matt,
    figtuple = (fig, ax), 
)

In [ ]:
# Tutorial note: Configure project directories, file paths, and SISepuede file structure. 
#df_peru.to_csv(path_data_built, encoding = "UTF-8", index = None, )




## 5. Working with generic transformations

The next cells explore and use the `transformation_general` helper to define
policy levers as transformations applied to the Peru input dataframe.


### What This Part Does

This section initializes the SISepuede machinery used to define and manage policy levers:

- Builds a **`transformations_peru`** directory on disk  
- Creates **Transformers**, **Transformations**, and **Strategies** objects  
- Prepares the system for **strategy-based scenario runs**  


### What a New User Needs to Know

- **`path_transformations`** is the directory where:
  - Strategy definitions  
  - Transformation specifications  
  - Helper files  
  are stored.

- This block constructs:
  - **`transformers`** → helper tools + input dataframe context  
  - **`transformations`** → complete set of transformation rules (policy levers)  
  - **`strategies`** → coherent collections of transformations representing scenarios  

- To use SISepuede for another country:
  - Change the **input dataframe**  
  - Optionally use a different transformations directory (e.g., `transformations_tza`)


## 6. Initializing Transformers, Transformations, and Strategies

The following cell(s) set up the Transformers / Transformations / Strategies
machinery used to define and manage policy levers for Peru. If you have already 
constructed the A6 experiments, you may not need to re-build experimental strategies.



### What a New User Needs to Know

- **Transformers**  
  Provide helper vectors and functions that allow transformations to apply smoothly across time, categories, and regions.

- **Transformations**  
  Represent *the rules* used to modify input data (e.g., scaling emissions, ramping fuel use, modifying forestry assumptions).

- **Strategies**  
  Bundle these transformations into a named scenario.  
  For Article 6, this could be:
  - A mitigation strategy  
  - A credit-selling country scenario  
  - An A6 cooperative implementation pathway  

- **This section is essential** if the goal is to run an Article 6 scenario, replicate past analyses, or generate new policy pathways.


### What Might Need Editing

If applying this workflow to another country or scenario:

- **`path_transformations`**  
  Should point to a country-specific directory (e.g., `transformations_chl`).

- **Input dataframe** (`df_peru`)  
  Should be swapped for the regional equivalent.

- **Strategy IDs and transformation specs**  
  May need to be updated depending on how many levers or policies your scenario requires.

This pattern, however, stays the same across all SISepuede regions and use cases.

In [6]:
# initialize a transformations set--will only run if the directory for transformations does not exist
# NOTE: you can always "start anew" with this


def get_transformations(
    path_transformations: pathlib.Path,
    df_inputs: pd.DataFrame,
) -> Tuple:
    """Using the path to transformations, build a transformations and strategies object.

    Returns a tuple of the form:

        (
            transformations,
            transformers,
            strategies
        )

        
    Function Arguments
    ------------------
    path_transformations : pathlib.Path
        Path to transformations directory usd to define transformations and strategies
    df_inputs : pd.DataFrame
        Base data DataFrame to modify with transformations
    """
    if not path_transformations.is_dir():
        print(f"Creating directory and instantiating default transformations at '{path_transformations}'...")
        trf.instantiate_default_strategy_directory(
            transformers,
            path_transformations,
        )
    
    
    # read in the transformations and assign
    transformations = tmts.Transformations(
        path_transformations,
        df_input = df_inputs,
    )
    
    transformers = transformations.transformers
    
    # initialize a strategies objet
    strategies = trf.Strategies(
        transformations,
        export_path = "transformations",
        prebuild = True,
    )

    
    out = (
        transformers,
        transformations,
        strategies
    )

    return out



# run the function--we'll use this later, after we build the A6 strategies--to keep it cleaner
(
    transformers,
    transformations,
    strategies
) = get_transformations(
    path_transformations,
    df_peru,
)



## 7. Article 6 Transformations

### What This Section Is Doing

This block constructs the **set of transformations** used for an Article 6 (A6) scenario. 


#### **YOU ONLY NEED TO RUN THIS ONCE AFTER INITIALIZING THE BASIC STRATEGIES/TRANSFORMATIONS DIRECTORY**

This block will add rows to the strategy attribute table, which you do not want to do indefinitely. The rows that are added are dependent on the transformations/strategies that you want to evaluate.


### Why This Matters for Article 6

Article 6 analysis requires:

- A **baseline** (e.g., business-as-usual or NDC)
- A **counterfactual scenario** modified by levers (representing Article 6 participation)

This block is where the machinery is built to define **those levers** in a modular, transparent way.

That makes this section a **core part of scenario construction**, unlike earlier diagnostics, which were mainly sanity checks.







In [ ]:

# SPECIFY TRANSFORMATION CODES THAT ARE TO BE EVALUATED ON/OFF AGAINST A BASELINE (SEE strats_to_build BELOW)
transformations_build = [
    "TX:AGRC:DEC_CH4_RICE",
    "TX:ENTC:TARGET_RENEWABLE_ELEC",
    "TX:INEN:INC_EFFICIENCY_ENERGY",
    "TX:TRNS:SHIFT_FUEL_LIGHT_DUTY",
    "TX:TRNS:SHIFT_MODE_FREIGHT",
    "TX:LNDU:DEC_DEFORESTATION",
    "TX:LNDU:INC_REFORESTATION",
    "TX:LVST:DEC_ENTERIC_FERMENTATION",
    "TX:SOIL:DEC_N_APPLIED",
    "TX:IPPU:DEC_HFCS"
]

##  INITIALIZE KEY ELEMENTS

# some basic vars
delim = "|"
codes = []
descriptions = []
ids = []
names = []
specs = []

# set the new id number to start at
id_num = max(strategies.attribute_table.key_values) + 1

# set a prefix for A6 strategies
_PREFIX_PFLO = "PFLO:"
_PREFIX_PFLO_A6 = f"{_PREFIX_PFLO}A6_"

# reference strategy and associated specifications
dict_strat_builds = {
    "PFLO:ALL": (1, 0), 
    "PFLO:NDC_2020": (1, 1),
}
dict_strat_pack = {}





#
strats_to_build = ["PFLO:ALL", "PFLO:NDC_2020"]

for code in strats_to_build:
    # get transformaion information
    strat_ref = strategies.get_strategy(code, )
    all_transformations_ref = strat_ref.get_transformation_list(
        strat_ref.transformation_specification,
        transformations,
    )
    all_transformation_codes_ref = [x.code for x in all_transformations_ref]

    # add to dict
    tup = (strat_ref, all_transformations_ref, all_transformation_codes_ref, )
    dict_strat_pack.update({code: tup, })



# map transformer codes to all transformations
dict_transformer_code_to_transformations = transformations.get_transformation_codes_by_transformer_code()




##  ITERATE OVER TRANSFORMATIONS THAT NEED TO BE ADDED/REMOVED FROM BASE REF

for strat_build, strat_pack in dict_strat_pack.items():

    builds = dict_strat_builds.get(strat_build)
    (strat_ref, all_transformations_ref, all_transformation_codes_ref, ) = strat_pack

    
    for transformation_code in transformations_build:
    
        # check for transformations in NDC
        transformation = transformations.get_transformation(transformation_code)
        if transformation is None:
            raise RuntimeError(f"Transformation code {transformation_code} not found.")
    
    
        # get transformer and check for existing transformation in reference strategy with that code
        all_transformations_with_code = dict_transformer_code_to_transformations.get(
            transformation.transformer_code, 
        )
    
        tfmns_with_trfr_in_strat_ref = set(all_transformations_with_code) & set(all_transformation_codes_ref)
        if len(tfmns_with_trfr_in_strat_ref) == 0: continue
    
        # transformation to replace with current one (for 100%) and remove (for 0%)
        transformation_remove = list(tfmns_with_trfr_in_strat_ref)[0]
    
    
        ## build transformation WITH 100% implementation
        transformation_specification_0 = sorted([x for x in all_transformation_codes_ref if x != transformation_remove])
        transformation_specification_100 = sorted(transformation_specification_0 + [transformation_code])
        
        transformation_specification_0_str = delim.join(transformation_specification_0)
        transformation_specification_100_str = delim.join(transformation_specification_100)

        # build the 0?
        if builds[0] == 1:
            code = transformation_remove.replace(":", "")
            code = f"{strat_build}_NO_{code}"
            code = code.replace(_PREFIX_PFLO, _PREFIX_PFLO_A6)
            name = f"A6 {strat_build} without {transformation_remove}"

            # append
            codes.append(code)
            descriptions.append(f"{name} for A6")
            ids.append(id_num)
            names.append(name)
            specs.append(transformation_specification_0_str)

            id_num += 1


        # build the 100?
        if builds[1] == 1:
            code = transformation_code.replace(":", "")
            code = f"{strat_build}_WITH_{code}"
            code = code.replace(_PREFIX_PFLO, _PREFIX_PFLO_A6)
            name = f"A6 {strat_build} with {transformation_code}"

            # append
            codes.append(code)
            descriptions.append(f"{name} replacing {transformation_remove} for A6")
            ids.append(id_num)
            names.append(name)
            specs.append(transformation_specification_100_str)

            id_num += 1




df_additional_strategies = pd.DataFrame(
    {
        strategies.attribute_table.key: ids,
        strategies.field_baseline_strategy: np.zeros(len(specs)).astype(int),
        strategies.field_description: descriptions,
        strategies.field_strategy_code: codes,
        strategies.field_strategy_name: names,
        strategies.field_transformation_specification: specs,
    }
)


# appends new stratgies to current table
attr_strat_out = strategies.attribute_table.table
attr_strat_out = sf._concat_df(
    [
        attr_strat_out,
        df_additional_strategies[attr_strat_out.columns]
    ]
)

## IMPORTANT! Export the new strategy attributes file

- This will export a new strategy attributes file
- Careful doing this--it shouldn't harm any strategies you've built otherwise, but it's always good to make a backup (hello GitHub!) before you export in case there are issues
- after exporting, we'll have to reload the transformations object

In [ ]:

sf._write_csv(
    attr_strat_out,
    strategies.path_strategy_definition,
)

# Now, reinstantiate strategies and transformations to reflect the experimental strategies we just built

In [ ]:
# run the function--we'll use this later, after we build the A6 strategies--to keep it cleaner
(
    transformers,
    transformations,
    strategies
) = get_transformations(
    path_transformations,
    df_peru,
)




## Now, set the strategies we want to run for our experiment

### Select the strategies that we need to run

- This is an example that focuses on (a) varying rice up and down and (b) the A6 on/off runs defined above
- If you are evaluating the performance of another individual transformation, you'll need to change `AGRC:DEC_CH4_RICE` to identify other strategies

###  setup ids that we want to build
- We will define the strategies that are associated with the A6 analysis
  `strat_ids_a6` is a list of these strategy ids to run in the experiment

###  build to template
- Note: we cannot use a `SISEPUEDE` object successfully to run the strategies unless we rebuild the templates using `strategies.build_strategies_to_templates`

In [ ]:
tab = strategies.attribute_table.table

# demonstrate A6 runs
strats_run = ["BASE", "PFLO:ALL", "PFLO:NDC_2020"]
strats_run += [x for x in list(tab[strategies.field_strategy_code]) if x.startswith("AGRC:DEC_CH4_RICE")]
strats_run += [x for x in tab["strategy_code"].to_numpy() if x.startswith("PFLO:A6")]
strat_ids_a6 = sorted([strategies.get_strategy(x).id_num for x in strats_run])

# optional code to build strategy ides for TORNADO plot (one by one on, singleton strategies)
"""
##  GET IN
strat_names_singleton = [
    x for x in strategies.attribute_table.table[strategies.field_strategy_name].to_numpy() 
    if x.startswith("Singleton")
]

# init
all_strategies_singleton = [strategies.baseline_id]
for strat in strategies.all_strategies:
    nm = strategies.get_strategy(strat).name
    if nm in strat_names:
        all_strategies_singleton.append(strat)
"""



# build templates--trajectory groups are not important right now
examples = sxl.SISEPUEDEExamples()
df_vargroups = examples("variable_trajectory_group_specification")
strategies.build_strategies_to_templates(
    include_simplex_group_as_trajgroup = False,
    strategies = strat_ids_a6,
)


# Next, build a SISEPUEDE object and load strategies
- This is what we'll use to actually run the analysis

In [7]:
import sisepuede as si

ssp = si.SISEPUEDE(
    "calibrated",
    db_type = "csv",
    #id_str = "sisepuede_run_2025-10-23T16:31:38.101072",
    initialize_as_dummy = False,
    regions = [_REGION_NAME],
    strategies = strategies,
    #try_exogenous_xl_types_in_variable_specification = True,
)





2025-11-13 02:58:47,885 - INFO - Successfully initialized SISEPUEDEFileStructure.
2025-11-13 02:58:47,886 - WARNING - Missing key dict_dimensional_keys: key time_series not found. Tables that rely on the time_series will not have index checking.
2025-11-13 02:58:47,887 - INFO - 	Setting export engine to 'csv'.
2025-11-13 02:58:47,887 - WARNING - No index fields defined. Index field values will not be checked when writing to tables.
2025-11-13 02:58:47,887 - INFO - Successfully instantiated table ANALYSIS_METADATA
2025-11-13 02:58:47,888 - WARNING - No index fields found in ATTRIBUTE_DESIGN. Initializing index fields.
2025-11-13 02:58:47,888 - INFO - Successfully instantiated table ATTRIBUTE_DESIGN
2025-11-13 02:58:47,888 - WARNING - No index fields found in ATTRIBUTE_LHC_SAMPLES_EXOGENOUS_UNCERTAINTIES. Initializing index fields.
2025-11-13 02:58:47,889 - INFO - Successfully instantiated table ATTRIBUTE_LHC_SAMPLES_EXOGENOUS_UNCERTAINTIES
2025-11-13 02:58:47,889 - WARNING - No index fi

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


Precompiling NemoMod...
Info Given NemoMod was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
   2032.8 ms  ? NemoMod
[ Info: Precompiling NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72] 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.
2025-11-13 02:59:52,534 - INFO - Successfully initialized JuMP optimizer from solver module HiGHS.
2025-11-13 02:59:52,549 - INFO - Successfully initialized SISEPUEDEModels.
2025-11-13 02:59:52,555 - INFO - Table ANALYSIS_METADATA successfully written to /Users/usuario/git/sisepuede/sisepuede/out/sisepuede_ru

## 8. Running example strategies via a scenario manager (optional)

These cells illustrate how to use a higher-level scenario manager (`ssp`) to
run predefined strategies such as `strat_base()` and `strat_a6()`. They are
useful examples but not strictly required to construct the core Peru inputs.

### What a New User Needs to Know

**`SISEPUEDEModels`** is the top-level **model runner** in SISepuede:

- It **orchestrates all sector models** (energy, AFOLU, industry, transport, waste, etc.).  
- It uses:
  - **Model metadata** (`matt`)  
  - **Reference files** from Julia and NemoMod (via `file_struct`)  

### For Strategy Runs

To run **scenario-based** (rather than baseline) simulations:

- Replace **`df_peru`** with a **transformed input dataframe** created using one or more policy levers or a defined strategy.
- Alternatively, use the **scenario manager approach** shown earlier, such as:
  - `strat_base()`
  - `strat_a6()`

This process allows you to generate, compare, and analyze **multiple scenarios** that all originate from the same base input dataset.



In [9]:

dict_run = {
    ssp.key_future: [0],
    ssp.key_design: [0],
    ssp.key_strategy: strat_ids_a6,#[0:1],
}

#primaries = [186186, 187187, 188188, 189189]
# we'll save inputs since we're doing a small set of runs 
dict_successful = ssp(
    dict_run, 
    force_overwrite_existing_primary_keys = True,
    save_inputs = True,
)

2025-11-13 02:59:59,492 - INFO - 
***	STARTING REGION peru	***

2025-11-13 03:00:01,164 - INFO - Trying run primary_id = 0 in region peru
2025-11-13 03:00:01,165 - INFO - Running AFOLU model
2025-11-13 03:00:01,406 - INFO - AFOLU model run successfully completed
2025-11-13 03:00:01,406 - INFO - Running CircularEconomy model
2025-11-13 03:00:01,432 - INFO - CircularEconomy model run successfully completed
2025-11-13 03:00:01,433 - INFO - Running IPPU model
2025-11-13 03:00:01,476 - INFO - IPPU model run successfully completed
2025-11-13 03:00:01,477 - INFO - Running Energy model (EnergyConsumption without Fugitive Emissions)
2025-11-13 03:00:01,487 - DEBUG - Missing elasticity information found in 'project_energy_consumption_by_fuel_from_effvars': using specified future demands.
2025-11-13 03:00:01,539 - INFO - EnergyConsumption without Fugitive Emissions model run successfully completed
2025-11-13 03:00:01,539 - INFO - Running Energy model (Electricity and Fuel Production: trying to ca

2025-13-Nov 03:00:01.830 Opened SQLite database at /Users/usuario/git/sisepuede/sisepuede/tmp/nemomod_intermediate_database.sqlite.
2025-13-Nov 03:00:01.975 Added NEMO structure to SQLite database at /Users/usuario/git/sisepuede/sisepuede/tmp/nemomod_intermediate_database.sqlite.
2025-13-Nov 03:00:17.358 Started modeling scenario. NEMO version = 2.2.0, solver = HiGHS.


2025-11-13 03:02:05,629 - INFO - NemoMod ran successfully with the following status: OPTIMAL
2025-11-13 03:02:05,636 - INFO - EnergyProduction model run successfully completed
2025-11-13 03:02:05,637 - INFO - Running Energy (Fugitive Emissions)
2025-11-13 03:02:05,657 - INFO - Fugitive Emissions from Energy model run successfully completed
2025-11-13 03:02:05,658 - INFO - Appending Socioeconomic outputs
2025-11-13 03:02:05,662 - INFO - Socioeconomic outputs successfully appended.
2025-11-13 03:02:05,665 - INFO - Model run for primary_id = 0 successfully completed in 124.5 seconds (n_tries = 1).
2025-11-13 03:02:05,668 - INFO - Trying run primary_id = 1001 in region peru
2025-11-13 03:02:05,669 - INFO - Running AFOLU model


2025-13-Nov 03:00:17.831 Started optimizing following years: [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011].
2025-13-Nov 03:00:40.339 Finished optimizing following years: [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011].
2025-13-Nov 03:00:40.386 Started optimizing following years: [1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035].
2025-13-Nov 03:02:05.511 Finished optimizing following years: [1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035].
set ['emission_co2e_co2_entc_bmass_processing_and_refinement_fp_hydrogen_gasification'] = 0 in energy production. FIX WITH NEW fp_hydrogen_gasification_biomass TECH.


2025-11-13 03:02:05,902 - INFO - AFOLU model run successfully completed
2025-11-13 03:02:05,902 - INFO - Running CircularEconomy model
2025-11-13 03:02:05,926 - INFO - CircularEconomy model run successfully completed
2025-11-13 03:02:05,926 - INFO - Running IPPU model
2025-11-13 03:02:05,966 - INFO - IPPU model run successfully completed
2025-11-13 03:02:05,966 - INFO - Running Energy model (EnergyConsumption without Fugitive Emissions)
2025-11-13 03:02:05,977 - DEBUG - Missing elasticity information found in 'project_energy_consumption_by_fuel_from_effvars': using specified future demands.
2025-11-13 03:02:06,024 - INFO - EnergyConsumption without Fugitive Emissions model run successfully completed
2025-11-13 03:02:06,025 - INFO - Running Energy model (Electricity and Fuel Production: trying to call Julia)


...

2025-11-13 07:27:28,247 - INFO - Table MODEL_OUTPUT successfully appended to /Users/usuario/git/sisepuede/sisepuede/out/sisepuede_run_2025-11-13T02;58;47.555474/sisepuede_run_2025-11-13T02;58;47.555474_output_database/MODEL_OUTPUT.csv.
2025-11-13 07:27:28,642 - INFO - Table MODEL_INPUT successfully appended to /Users/usuario/git/sisepuede/sisepuede/out/sisepuede_run_2025-11-13T02;58;47.555474/sisepuede_run_2025-11-13T02;58;47.555474_output_database/MODEL_INPUT.csv.
2025-11-13 07:27:28,643 - INFO - Trying run primary_id = 200200 in region peru
2025-11-13 07:27:28,644 - INFO - Running AFOLU model
2025-11-13 07:27:28,866 - INFO - AFOLU model run successfully completed
2025-11-13 07:27:28,867 - INFO - Running CircularEconomy model
2025-11-13 07:27:28,889 - INFO - CircularEconomy model run successfully completed
2025-11-13 07:27:28,890 - INFO - Running IPPU model
2025-11-13 07:27:28,928 - INFO - IPPU model run successfully completed
2025-11-13 07:27:28,928 - INFO - Running Energy model (Ene

2025-13-Nov 07:27:29.601 Started modeling scenario. NEMO version = 2.2.0, solver = HiGHS.
2025-13-Nov 07:27:29.647 Started optimizing following years: [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011].


2025-11-13 07:28:53,903 - INFO - NemoMod ran successfully with the following status: OPTIMAL
2025-11-13 07:28:53,909 - INFO - EnergyProduction model run successfully completed
2025-11-13 07:28:53,909 - INFO - Running Energy (Fugitive Emissions)
2025-11-13 07:28:53,931 - INFO - Fugitive Emissions from Energy model run successfully completed
2025-11-13 07:28:53,931 - INFO - Appending Socioeconomic outputs
2025-11-13 07:28:53,936 - INFO - Socioeconomic outputs successfully appended.
2025-11-13 07:28:53,938 - INFO - Model run for primary_id = 200200 successfully completed in 85.29 seconds (n_tries = 1).
2025-11-13 07:28:53,939 - INFO - 
***	 REGION peru COMPLETE	***

2025-11-13 07:28:54,011 - INFO - Table MODEL_OUTPUT successfully appended to /Users/usuario/git/sisepuede/sisepuede/out/sisepuede_run_2025-11-13T02;58;47.555474/sisepuede_run_2025-11-13T02;58;47.555474_output_database/MODEL_OUTPUT.csv.
2025-11-13 07:28:54,013 - INFO - Table ATTRIBUTE_PRIMARY successfully written to /Users/usua

2025-13-Nov 07:27:47.173 Finished optimizing following years: [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011].
2025-13-Nov 07:27:47.205 Started optimizing following years: [1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035].
2025-13-Nov 07:28:53.813 Finished optimizing following years: [1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035].
set ['emission_co2e_co2_entc_bmass_processing_and_refinement_fp_hydrogen_gasification'] = 0 in energy production. FIX WITH NEW fp_hydrogen_gasification_biomass TECH.


2025-11-13 07:28:54,119 - INFO - Table MODEL_INPUT successfully appended to /Users/usuario/git/sisepuede/sisepuede/out/sisepuede_run_2025-11-13T02;58;47.555474/sisepuede_run_2025-11-13T02;58;47.555474_output_database/MODEL_INPUT.csv.


In [295]:
# check primaries that ran
primaries = dict_successful.get("peru")


In [301]:
##  check the primaries that ran in table for
attribute_prim = ssp.odpt_primary.get_indexing_dataframe_from_primary_key(primaries)

,primary_id,design_id,strategy_id,future_id
1,1001,0,1000,0
2,2002,0,1001,0
3,3003,0,1002,0
4,4004,0,1003,0
5,5005,0,1004,0
6,6006,0,1005,0
7,7007,0,1006,0
8,8008,0,1007,0
9,9009,0,1008,0
10,10010,0,1009,0


###  The next block allows you to read in the inputs, outputs, and generate a summary set of files for use in Tableau  other visualization software

In [11]:
df_out_tmp = ssp.read_output(None)
df_in_tmp = ssp.read_input(None)
ssp.generate_summary_files(None, export = True, );

## 9. You can visualize outputs in stakcs
- Explore the output data in `df_out` or inputs in `df_in`
- `svp.plot_emissions_stack` will automatically plot subsector total emissions contained in the output dataframe
- May want to set `id_primary` to only plot a single primary id

In [ ]:
id_primary_plot = 0       # set a primary id to plot
fig, ax = plt.subplots(figsize = (11, 7))
svp.plot_emissions_stack(
    df_out[
        df_out[ssp.key_primary].isin([id_primary_plot])
    ],
    matt,
    figtuple = (fig, ax),
)
ax.legend()


### You can also plot individual variables

In [ ]:
# Tutorial note: Exploratory inspection, printing, or plotting for validation. 
modvar = matt.get_variable("Land Use Area", )
fig, ax = plt.subplots(figsize = (12, 8))
df_run_enerprod[modvar.fields].plot.area(cmap = "tab20", ax = ax, )
ax.legend(loc = "upper right")
